# Working model with dumb preprocessing
A version of `keras-model-2` that allows us to not use our pickle (`.p`) files.


In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.layers.recurrent import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

from preprocess import *
from util import *

Using Theano backend.


In [2]:
data, labels, min_song_length = loadUnprocessedSongs(num_files=2)

  0%|          | 0/2 [00:00<?, ?it/s]

Listing files
Loading files
Calculating minimum song length
Separating voices


100%|██████████| 2/2 [00:03<00:00,  2.05s/it]

Getting soprano part (input data)
Getting bass part (output labels)


In [3]:
data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

(2, 120)
(2, 120, 89)


In [4]:
np.min(data)

19

In [5]:
#TODO: shift data down to 0-89 instead of 19-...
NUM_POSSIBLE_ACTIONS = len(pitchToTuple(SILENCE))
EMBEDDING_DIMENSIONS=12
#MIN_SONG_LENGTH=200 #TODO figure out/look up

model = Sequential()

model.add(Embedding(NUM_POSSIBLE_ACTIONS, EMBEDDING_DIMENSIONS, input_length=min_song_length))

model.add(LSTM(EMBEDDING_DIMENSIONS, return_sequences=True))

#model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(NUM_POSSIBLE_ACTIONS)))

print(model.output_shape)
model.add(Activation('softmax'))

# Binary crossentropy?
model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['accuracy'])



(None, 120, 89)


In [6]:
model.fit(data, labels, nb_epoch=10, batch_size=1)

Epoch 1/10
2/2 [==============================] - 0s - loss: 0.0111 - acc: 0.0042     
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.0111 - acc: 0.0125     
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.0111 - acc: 0.0208     
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.0111 - acc: 0.0458     
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.0111 - acc: 0.0458     
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.0110 - acc: 0.0458     
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.0110 - acc: 0.0458     
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.0110 - acc: 0.0458     
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.0110 - acc: 0.0458     
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.0110 - acc: 0.0458     


In [7]:
output=model.predict(data[0, np.newaxis])
np.array([np.argmax(output[0, time, :]) for time in range(output.shape[2])])+LOWEST_PITCH

array([20, 20, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
       52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
       52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
       52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
       52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
       52, 52, 52, 52])